In [169]:
import pandas as pd
import re
import numpy as np
import requests
import html
import datetime
import json
import time
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer

In [170]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sharonmoscato/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [67]:
books_df = pd.read_csv('books.csv', sep=";", error_bad_lines=False, encoding="latin-1")
ratings_df = pd.read_csv('ratings.csv', sep=";", error_bad_lines=False, encoding="latin-1")
users_df = pd.read_csv('users.csv', sep=";", error_bad_lines=False, encoding="latin-1")

/var/folders/nm/8604s05d4271f1m5s1tk39rh0000gn/T/ipykernel_25448/1376693211.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books_df = pd.read_csv('books.csv', sep=";", error_bad_lines=False, encoding="latin-1")
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 

In [68]:
print(books_df.shape)
print(ratings_df.shape)
print(users_df.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


# Exploration books data set

In [69]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [70]:
books_df.head(10)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...
7,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...
8,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,http://images.amazon.com/images/P/0679425608.0...,http://images.amazon.com/images/P/0679425608.0...,http://images.amazon.com/images/P/0679425608.0...
9,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...


In [71]:
# Unnecessary columns dropped
books_df = books_df.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'])
books_df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press


In [72]:
# Function for check if ISBN are valid, defined with regex isbn10 and isbn13 and created a function to detect suspicious ISBNs
def is_valid_isbn(isbn):
    isbn = re.sub(r'[-\s]', '', isbn)
    isbn_10_pattern = re.compile(r'^\d{9}[\dXx]$')
    isbn_13_pattern = re.compile(r'^\d{13}$')
    return bool(isbn_10_pattern.match(isbn)) or bool(isbn_13_pattern.match(isbn))

invalid_isbn_books = books_df[~books_df['ISBN'].apply(is_valid_isbn)]
invalid_isbn_books 

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
6291,B00009ANY9,Cane River,Lalita Tademy,2001,Warner Books
10272,B0000A2U93,Carmilla,Joseph Sheridan Le Fanu,0,Soft Editions Ltd
11185,B0000633PU,The Story of Aladdin and the Wonderful Lamp,S. Lane Poole,0,Renaissance eBooks
13862,B00007FYKO,Bloodcurdling Tales of Horror and the Macabre:...,H. P. Lovecraft,0,Ballantine
13865,B00009APKU,Moby Dick,Herman Melville,0,"Outrigger Publishing, LLC"
...,...,...,...,...,...
252068,B00013AX9E,Modern Contemporary: Aspects of Art at MoMA Si...,P Ed Varnedoe/Antonelli Kick,2000,Museum of Modern Arts
256444,B00009NDBF,Help Yourself: Celebrating the Rewards of Resi...,David J. Pelzer,1900,E P Dutton
262551,B000023VWR,The Postman Always Rings Twice,James Mallahan Cain,1996,Otto Penzler
264207,B00007CWGV,Madonna,Andrew Morton,2001,St. Martin's Press


In [73]:
books_df = books_df[books_df['ISBN'].apply(is_valid_isbn)].reset_index(drop=True)
books_df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271238,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271239,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
271240,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271241,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press


In [74]:
books_df['Year-Of-Publication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 2038, 2026, 1944, 1917, 1901, 2010,
       1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980', '197

There are some suspicious Year of publication as 0 or higher than current year, decided to replace the value with none\
Invalid Year as DK Publishing Inc and Gallimard needs to be shift to publisher

In [75]:
invalid_year_rows = books_df[~books_df['Year-Of-Publication'].astype(str).str.isnumeric()]
invalid_year_rows

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
209443,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
220636,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...
221583,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


In [76]:
indxtoshift = invalid_year_rows.index
for idx in indxtoshift:
    books_df.at[idx, 'Publisher'] = books_df.at[idx, 'Year-Of-Publication']
    books_df.at[idx, 'Year-Of-Publication'] = books_df.at[idx, 'Book-Author']
    books_df.at[idx, 'Book-Author'] = None 
books_df.loc[indxtoshift]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
209443,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",None,2000,DK Publishing Inc
220636,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",None,2003,Gallimard
221583,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",None,2000,DK Publishing Inc


In [77]:
# The column has combination of str, and int. Converting all the value to int 
books_df['Year-Of-Publication'] = pd.to_numeric(books_df['Year-Of-Publication'], errors='coerce') 
books_df['Year-Of-Publication'] = books_df['Year-Of-Publication'].astype(pd.Int64Dtype())

# Replacing value that are equal to 0 or higher than curren year with na
books_df.loc[books_df['Year-Of-Publication'] == 0, 'Year-Of-Publication'] = pd.NA
books_df.loc[books_df['Year-Of-Publication'] > int(datetime.date.today().strftime('%Y')), 'Year-Of-Publication'] = pd.NA

In [78]:
books_df['author_name_length'] = books_df['Book-Author'].astype(str).apply(len)
sorted_books_df = books_df.sort_values(by='author_name_length', ascending=False)
sorted_books_df[['Book-Author', 'author_name_length']].head()

,Book-Author,author_name_length
216006,"Dale D. Johnson, Theodore Clymer, Roselmina In...",143
42791,Based on the Larger Work'the Book of Garden Fl...,122
175566,"Staceyann Chin, Dot Antoniades, Aileen Reyes, ...",107
92419,Center for Chemical Process Safety for the Ame...,70
192700,United States. Agricultural Research Service. ...,70


In [79]:
books_df.loc[219783][1]

"Chomsky's Universal Grammar: An Introduction"

Book-Author, Book-Title and Publisher will need to be cleaned of whitespace, Html entities etc - this will be done later once the datasets have been merged.

In [80]:
# Deleted unnecessary column created in the above cell
books_df = books_df.drop(columns=['author_name_length'])

# Exploration ratings data set

In [81]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [82]:
ratings_df[~ratings_df['ISBN'].apply(is_valid_isbn)]

,User-ID,ISBN,Book-Rating
9,276745,342310538,10
40,276762,342662429,0
54,276762,B0000BLD7X,0
55,276762,N3453124715,4
247,276856,20103389,0
...,...,...,...
1149238,276641,033031582,5
1149358,276670,00969754916,7
1149428,276670,15655122046,6
1149439,276670,55305663,6


Checked ISBN in ratings df and there is 10159 rows with non valid ISBN, decided to drop them.

In [83]:
ratings_df = ratings_df[ratings_df['ISBN'].apply(is_valid_isbn)].reset_index(drop=True)

In [84]:
ratings_df["Book-Rating"].unique()

array([ 0,  5,  3,  6,  8,  7,  9, 10,  4,  1,  2])

# Exploration users dataset

In [85]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [86]:
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [87]:
users_df['Age'].describe()

count    168096.000000
mean         34.751434
std          14.428097
min           0.000000
25%          24.000000
50%          32.000000
75%          44.000000
max         244.000000
Name: Age, dtype: float64

The maximum and minimum of the values (0 and 244) in the age column indicate that the age column contains incorrect values

In [88]:
type(users_df['Age'][0])

numpy.float64

In [89]:
users_df['Age'] = users_df['Age'].astype(pd.Int64Dtype())

In [90]:
users_df['Age'].unique()

<IntegerArray>
[<NA>,   18,   17,   61,   26,   14,   25,   19,   46,   55,
 ...
  107,  157,  111,  146,  118,  220,  143,  140,  189,  127]
Length: 166, dtype: Int64

In [91]:
users_df.loc[(users_df['Age'] < 6) | (users_df['Age'] > 99), 'Age'] = np.nan
users_df['Age'] = users_df['Age'].astype(pd.Int64Dtype())

In [92]:
users_df[users_df['Age'].isnull()]

,User-ID,Location,Age
0,1,"nyc, new york, usa",<NA>
2,3,"moscow, yukon territory, russia",<NA>
4,5,"farnborough, hants, united kingdom",<NA>
6,7,"washington, dc, usa",<NA>
7,8,"timmins, ontario, canada",<NA>
...,...,...,...
278849,278850,"sergnano, lombardia, italy",<NA>
278853,278854,"portland, oregon, usa",<NA>
278855,278856,"brampton, ontario, canada",<NA>
278856,278857,"knoxville, tennessee, usa",<NA>


Apparently ~40% do not seem to have age information filled in

## Merge data frames and clean data

In [93]:
# Function to check if there are white spaces, HTML entits and HTML tags or double spaces
def analyze_text_issues(text):
    double_space = bool(re.search(r'  ', text))
    html_tag = bool(re.search(r'<.*?>', text))
    html_entity = text != html.unescape(text)
    extra_whitespace = text != text.strip()

    return double_space, html_tag, html_entity, extra_whitespace

# Function which use function analyze_text_issues in columns
def summarize_column_issues(df, columns_to_inspect):
    summary = {}
    for column in columns_to_inspect:
        results = df[column].astype(str).dropna().apply(analyze_text_issues)
        summary[column] = {
            'double_spaces': results.apply(lambda x: x[0]).sum(),
            'html_tags': results.apply(lambda x: x[1]).sum(),
            'html_entities': results.apply(lambda x: x[2]).sum(),
            'extra_whitespace': results.apply(lambda x: x[3]).sum()
        }
    return summary

In [94]:
# Function to clean data from white spaces, HTML entits and HTML tags or double spaces
def clean_text_data(text):
    previous_text = ""
    while previous_text != text:
        previous_text = text
        text = re.sub(r'<.*?>', '', text)
        text = text.replace('  ', ' ')
        text = html.unescape(text)
        text = text.strip()
    return text

In [95]:
# Merge user and ratings info to get one dataframe with user-ratings info.
user_ratings_df = pd.merge(ratings_df, users_df, on='User-ID', how='inner')

In [96]:
# Dataframe user_ratings_df grouped by ISBN and aggregating the information about age, book rating and user-id in list
# The final merge contains the main book dataframe with information about rating, age and user- id aggregated in list
isbn_user_ratings = user_ratings_df.groupby('ISBN').agg({
    'User-ID': list,
    'Book-Rating': list,
    'Age': list 
})
merged_df = pd.merge(books_df, isbn_user_ratings, on='ISBN', how='left')

In [97]:
merged_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,[2],[0],[18]
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"[8, 11400, 11676, 41385, 67544, 85526, 96054, ...","[5, 0, 8, 0, 8, 0, 0, 9, 9, 0, 8, 9, 7, 6]","[<NA>, 49, <NA>, <NA>, 30, 36, 29, <NA>, <NA>,..."
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"[8, 2954, 152827]","[0, 8, 7]","[<NA>, 71, 40]"
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"[8, 35704, 83160, 110912, 137688, 157969, 1691...","[0, 6, 0, 10, 0, 8, 0, 8, 9, 6, 0]","[<NA>, 53, 65, 36, 46, 30, <NA>, 47, 49, <NA>,..."
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,[8],[0],[<NA>]


In [98]:
columns_to_check = ['Book-Title', 'Book-Author', 'Publisher']
summary = summarize_column_issues(merged_df, columns_to_check)
summary

{'Book-Title': {'double_spaces': 2246,
  'html_tags': 0,
  'html_entities': 4894,
  'extra_whitespace': 47},
 'Book-Author': {'double_spaces': 1310,
  'html_tags': 0,
  'html_entities': 270,
  'extra_whitespace': 7},
 'Publisher': {'double_spaces': 2,
  'html_tags': 0,
  'html_entities': 15780,
  'extra_whitespace': 2}}

In [99]:
# Apply function clean_text_data on columns 'Book-Title', 'Book-Author', 'Publisher'
for column in ['Book-Title', 'Book-Author', 'Publisher']:
    merged_df[column] = merged_df[column].astype(str).apply(clean_text_data)
merged_df[['Book-Title', 'Book-Author', 'Publisher']].head()

,Book-Title,Book-Author,Publisher
0,Classical Mythology,Mark P. O. Morford,Oxford University Press
1,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada
2,Decision in Normandy,Carlo D'Este,HarperPerennial
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux
4,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton & Company


In [100]:
merged_df.isnull().sum()

ISBN                      0
Book-Title                0
Book-Author               0
Year-Of-Publication    4576
Publisher                 0
User-ID                1209
Book-Rating            1209
Age                    1209
dtype: int64

Several columns have empty values, for example Year-Of-Publication, Publisher can be checked to get data from the API.

## Getting data from API

In [101]:
# Function to create a list of query where each element has 100 unique ISBN, which meets the conditions of the API.
def create_list_query(merged_df):
    unique_isbn = merged_df['ISBN'].unique()
    isbn_list = unique_isbn.tolist()
    chunked_isbn_list = [isbn_list[i:i + 100] for i in range(0, len(isbn_list), 100)]
    list_query = []
    for chunk in chunked_isbn_list:
        isbn_str = ",".join(f"ISBN:{isbn}" for isbn in chunk)
        url = f"https://openlibrary.org/api/books?bibkeys={isbn_str}&jscmd=details&format=json"
        list_query.append(url)
    return list_query

In [102]:
# list_query = create_list_query(merged_df)

In [103]:
# Function to get data and continuosly save them in a new line in data.json file, if error will occure it will give the url and status code
def get_data(list_query):
    with open('data.json', 'a') as output_file:  
        for url in list_query:
                response = requests.get(url)
                if response.status_code == 200:
                    data = response.json()
                    json.dump(data, output_file)
                    output_file.write('\n')  # Newline character to separate JSON objects
                else:
                    print(f"Request failed for {url}. Status code: {response.status_code}")
                time.sleep(1)

In [104]:
#get_data(list_query)

In [105]:
def extract_book_data(file_path):
    all_books = []
    with open(file_path, 'r') as file:
        for line in file:
            json_obj = json.loads(line.strip())

            for isbn, book_info in json_obj.items():
                if isinstance(book_info, dict):
                    details = book_info.get('details', {})
                    
                    if isinstance(details, dict):
                        title = details.get('title')
                        number_of_pages = details.get('number_of_pages')
                        publish_date = details.get('publish_date')

                        publishers = details.get('publishers', [])
                        publisher = publishers[0] if publishers else None
                        
                        if isinstance(details.get('subjects', []), list):
                            subjects = details.get('subjects', [])
                        else:
                            subjects = []

                        if isinstance(details.get('genres', []), list):
                            genres = details.get('genres', [])
                        else:
                            genres = []

                        book_data = {
                            'ISBN': isbn,
                            'Title': title,
                            'Number_of_Pages': number_of_pages,
                            'Publisher': publisher,
                            'publish_date': publish_date,
                            'Subjects': ', '.join(subjects), 
                            'Genres': ', '.join(genres) 
                        }
                        all_books.append(book_data)

    return pd.DataFrame(all_books)

In [106]:
file_path = 'data.json' 
external_books_df = extract_book_data(file_path)
external_books_df.head(10)

,ISBN,Title,Number_of_Pages,Publisher,publish_date,Subjects,Genres
0,ISBN:034545104X,Flesh tones,310.0,Ballantine Books,2002,"Trials (Murder) -- Fiction., New York (N.Y.) -...",Fiction.
1,ISBN:0155061224,Rites of passage,404.0,Heinle & Heinle/Thomson Learning,2002,"College readers., Readers -- Maturation (Psych...","Problems, exercises, etc."
2,ISBN:0446520802,the notebook,NaN,"New York : Warner Books, 1996",1996,,
3,ISBN:052165615X,Help!,32.0,Cambridge University Press,1999,"Computers -- Fiction, Readers for new literates",Fiction.
4,ISBN:0521795028,The Amsterdam Connection,80.0,Cambridge University Press,"March 19, 2001","ELT audio-visual (video & audio cassettes), EL...",
5,ISBN:2080674722,Les particules élémentaires,393.0,Flammarion,1998,,
6,ISBN:038550120X,A Painted House,NaN,Doubleday,2001-02,,
7,ISBN:0425115801,Lightning,NaN,Berkley,1988,,
8,ISBN:0449006522,The Manhattan hunt club.,NaN,Ballantine Books,2002,,
9,ISBN:0553561618,Dark paradise.,528.0,Bantam,None,,


### Cleaning data from API

In [107]:
# Cleaning from ISBN:
external_books_df['ISBN'] = external_books_df['ISBN'].str.replace('ISBN:', '', regex=False)

In [108]:
# Page number from float to int
external_books_df['Number_of_Pages'] = external_books_df['Number_of_Pages'].astype(pd.Int64Dtype())

In [109]:
columns_to_check = ['Title', 'Subjects', 'Publisher', 'Genres']
summary = summarize_column_issues(external_books_df, columns_to_check)
summary

{'Title': {'double_spaces': 1376,
  'html_tags': 5,
  'html_entities': 5,
  'extra_whitespace': 1178},
 'Subjects': {'double_spaces': 1254,
  'html_tags': 2,
  'html_entities': 0,
  'extra_whitespace': 7},
 'Publisher': {'double_spaces': 19,
  'html_tags': 0,
  'html_entities': 1,
  'extra_whitespace': 25},
 'Genres': {'double_spaces': 9,
  'html_tags': 0,
  'html_entities': 0,
  'extra_whitespace': 9}}

In [110]:
for column in ['Title', 'Subjects', 'Publisher', 'Genres']:
    external_books_df[column] = external_books_df[column].astype(str).apply(clean_text_data)
external_books_df[['Title', 'Subjects', 'Publisher', 'Genres']].head()

,Title,Subjects,Publisher,Genres
0,Flesh tones,"Trials (Murder) -- Fiction., New York (N.Y.) -...",Ballantine Books,Fiction.
1,Rites of passage,"College readers., Readers -- Maturation (Psych...",Heinle & Heinle/Thomson Learning,"Problems, exercises, etc."
2,the notebook,,"New York : Warner Books, 1996",
3,Help!,"Computers -- Fiction, Readers for new literates",Cambridge University Press,Fiction.
4,The Amsterdam Connection,"ELT audio-visual (video & audio cassettes), EL...",Cambridge University Press,


In [111]:
# Convert the 'publish_date' column to datetime and keeping only the information about year
external_books_df['publish_date'] = pd.to_datetime(external_books_df['publish_date'], errors='coerce')
external_books_df['year'] = external_books_df['publish_date'].dt.year

/var/folders/nm/8604s05d4271f1m5s1tk39rh0000gn/T/ipykernel_25448/2286062309.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  external_books_df['publish_date'] = pd.to_datetime(external_books_df['publish_date'], errors='coerce')


In [112]:
external_books_df['year'] = external_books_df['publish_date'].dt.year

# Drop the original 'publish_date' column as we now have the year
external_books_df = external_books_df.drop('publish_date', axis=1)

# Return the first few rows of the modified DataFrame to check the result
external_books_df.head()

,ISBN,Title,Number_of_Pages,Publisher,Subjects,Genres,year
0,034545104X,Flesh tones,310,Ballantine Books,"Trials (Murder) -- Fiction., New York (N.Y.) -...",Fiction.,2002.0
1,0155061224,Rites of passage,404,Heinle & Heinle/Thomson Learning,"College readers., Readers -- Maturation (Psych...","Problems, exercises, etc.",2002.0
2,0446520802,the notebook,<NA>,"New York : Warner Books, 1996",,,1996.0
3,052165615X,Help!,32,Cambridge University Press,"Computers -- Fiction, Readers for new literates",Fiction.,1999.0
4,0521795028,The Amsterdam Connection,80,Cambridge University Press,"ELT audio-visual (video & audio cassettes), EL...",,2001.0


In [113]:
# Converting year value to int
external_books_df['year'] = external_books_df['year'].astype(pd.Int64Dtype())

In [114]:
# Filling information about year which is present in external_book_df and missing in merged_df
# It filled 3574 values
comparison_df = pd.merge(merged_df[['ISBN', 'Year-Of-Publication']], external_books_df[['ISBN', 'year']], on='ISBN', how='outer', indicator=True)
condition = (comparison_df['Year-Of-Publication'].isna()) & (comparison_df['year'].notna())
isbns_to_update = comparison_df.loc[condition, 'ISBN']
for isbn in isbns_to_update:
    new_year = comparison_df.loc[comparison_df['ISBN'] == isbn, 'year'].iloc[0]
    merged_df.loc[merged_df['ISBN'] == isbn, 'Year-Of-Publication'] = new_year


In [115]:
unique_genres = external_books_df['Genres'].unique()
unique_genres

array(['Fiction.', 'Problems, exercises, etc.', '', 'Fiction',
       'Juvenile fiction.', 'Biography.', 'Humor.', 'Interviews.',
       'Juvenile poetry., Poetry.', 'Juvenile fiction., Fiction.',
       'Guidebooks.', 'Biography', 'Exhibitions, In art',
       'Juvenile literature., Specimens.', 'Case studies., Biography.',
       'Catalogs., Biography.', 'Anecdotes.', 'Juvenile fiction',
       'Case studies.', 'Fiction., fiction.', 'Specimens.', 'Poetry',
       'Fiction., Juvenile fiction.', 'Juvenile literature.',
       'Folklore., Fiction.', 'Specimens., Fiction.',
       'Fiction., Novela juvenil.', 'Biography., Anecdotes.',
       'Case studies', 'Humor', 'Biography., Humor.', 'Poetry.',
       'Quotations, maxims, etc.', 'Humor., Parodies, imitations, etc.',
       'Drama.', 'Juvenile fiction., Juvenile literature.',
       'Fiction., Specimens.', 'Meditations.', 'Terms and phrases.',
       'Quotations.', 'Fiction., Adaptations.', 'Pictorial works.',
       'Juvenile literat

In [116]:
# Removing text "etc., etc and .", double spaces 
def clean_and_split_genres(genre):
    if pd.isna(genre):
        return genre 
    
    cleaned_genre = re.sub(r'\.', '', genre)
    cleaned_genre = re.sub(r'\betc\b\.?', '', cleaned_genre)
    cleaned_genre = re.sub(r'\s+', ' ', cleaned_genre)
    split_genres = [g.strip() for g in cleaned_genre.split(',') if g.strip()]
    
    return split_genres
external_books_df['Genres'] = external_books_df['Genres'].apply(clean_and_split_genres)

In [117]:
empty_genres_count = external_books_df[external_books_df['Genres'].apply(lambda x: not x)].shape[0]
empty_genres_count

236759

Unfortunatelly there are so many empty list for column 'Genres', that this columns can't be used.
The column 'Subject' contains many key words which can help to reccomend books


In [162]:
def clean_and_split(text):

    if pd.isna(text):
        return []

    text = text.lower()

    cleaned_text = re.sub(r'[^a-z0-9\s-]', ' ', text)  
    cleaned_text = re.sub(r'(?<!\w)-|-(?!\w)', ' ', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    word_list = cleaned_text.split()

    stop_words = set(stopwords.words('english'))
    word_list = [word for word in word_list if word not in stop_words]
    word_list = cleaned_text.split()

    return word_list


In [163]:
# Apply the function to both 'Subjects' and 'Genres' columns
external_books_df['Cleaned_Subjects'] = external_books_df['Subjects'].apply(clean_and_split)
external_books_df['Cleaned_Genres'] = external_books_df['Genres'].apply(clean_and_split)

In [166]:
external_books_df['Concatenated_S_G'] = external_books_df['Cleaned_Subjects'] + external_books_df['Cleaned_Genres']

In [171]:
def lemmatize_words(word_list):
    lemmatizer = WordNetLemmatizer()
    # Lemmatize each word in the list
    lemmatized_words = [lemmatizer.lemmatize(word) for word in word_list]
    return lemmatized_words


In [215]:
# Assuming 'Concatenated_S_G' is a list of words, you would apply the function like this:
external_books_df['Lemmatized_S_G'] = external_books_df['Concatenated_S_G'].apply(lemmatize_words)

In [206]:
# Used to check which genres are in our data to specify the known genres 
# non_empty_genres = external_books_df['Cleaned_Genres'].dropna().loc[external_books_df['Cleaned_Genres'] != '']
# unique_genres_set = set()
# for genre_list in external_books_df['Cleaned_Genres']:
#     if genre_list:  
#         unique_genres_set.update(genre_list)
# print(list(unique_genres_set))

In [306]:
known_genres = [
    'adventure', 'biography', 'fantasy', 'historical', 'horror', 'literary', 'mystery', 
    'mythology', 'non-fiction', 'philosophical', 'romance', 'satire', 'science', 'juvenile',
    'thriller', 'western', 'young', 'action', 'drama', 'erotica', 'memoir', 'crime', 
    'dystopian', 'self-help', 'travel', 'guide', 'anthology', 'classic', 'comedy', 
    'psychological', 'suspense', 'tragedy', 'fairy', 'folklore', 'legend', 'narrative', 
    'periodical', 'political', 'realistic', 'reference', 'religion', 'short', 'superhero', 
    'supernatural', 'textbook', 'urban', 'utopian', 'war', 'absurdist', 'alternate', 
    'coming-of-age', 'cookbook', 'diary', 'encyclopedia', 'epic', 'experimental', 'fable', 
    'fan', 'gothic', 'graphic', 'hard-boiled', 'historiography', 'humor', 'lab', 'magical', 
    'paranormal', 'picaresque', 'post-apocalyptic', 'stream-of-consciousness', 'sword', 
    'true', 'vampire', 'visionary', 'whodunit', 'non-fiction', 'biology', 'music', 'guidebook',
    'vocabularies', 'design', 'architecture', 'novela', 'archeology', 'tour', 'statistic',
    'anecdotes', 'guidebook', 'manual', 'history', 'child', 'study', 'work', 'dictionaries', 'humor',
    'handbook', 'pictorial', 'personal', 'poetry', 'interview', 'fiction', 'literature', 'guidebooks',
    'social', 'detective', 'life', 'fictitious'
]


In [309]:
# Function to check for matched from list know_genres in df with list comprehension
def find_genre_matches(lemmatized_genres, known_genres):
    match = [genre for genre in known_genres if genre in lemmatized_genres]
    return match

In [310]:
# Created new column category where are saved category of book in list appling function find_genre_matches
external_books_df['category'] = external_books_df['Lemmatized_S_G'].apply(lambda row: find_genre_matches(row, known_genres))

In [311]:
external_books_df['category'].apply(lambda x: len(x) == 0).sum()

118831

In [312]:
269364 - 118831


150533

In [314]:
(100/269364) * 150533

55.884602248258865

I have created a new column named 'category' that contains more accurate information regarding the categories of various books, presented in list format. This enhancement came after cleaning the 'Subject' and 'Genres' columns, followed by word splitting and the application of lemmatization techniques using a specific library. Subsequently, I established a list of prevalent book categories. Upon cross-verifying the presence of these categories within the text, I generated a new column. This column boasts a fill rate of approximately 55%. This new column will be used for book recommendation model more accorated.

## Final processing of merged dataframe

In [128]:
# Creating a new column named 'Publication-Range' whith a range of Year-Of-Publication by decade, it is by defualt left closed
bins = list(range(1900, 2030, 10))  
labels = [f"{i}-{i+10}" for i in bins[:-1]]

merged_df['Publication-Range'] = pd.cut(merged_df['Year-Of-Publication'], bins=bins, labels=labels, right=False)

In [129]:
merged_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age,Publication-Range
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,[2],[0],[18],2000-2010
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"[8, 11400, 11676, 41385, 67544, 85526, 96054, ...","[5, 0, 8, 0, 8, 0, 0, 9, 9, 0, 8, 9, 7, 6]","[<NA>, 49, <NA>, <NA>, 30, 36, 29, <NA>, <NA>,...",2000-2010
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"[8, 2954, 152827]","[0, 8, 7]","[<NA>, 71, 40]",1990-2000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"[8, 35704, 83160, 110912, 137688, 157969, 1691...","[0, 6, 0, 10, 0, 8, 0, 8, 9, 6, 0]","[<NA>, 53, 65, 36, 46, 30, <NA>, 47, 49, <NA>,...",1990-2000
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,[8],[0],[<NA>],1990-2000
